<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - Funk SVD F1 </h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [2]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, funksvd
from lenskit.algorithms.funksvd import FunkSVD
import pandas as pd
import numpy as np
np.seterr(divide='ignore', invalid='ignore')

ratings = pd.read_csv('D:\\Escritorio\\UM\\Tesis\\ML 1M\\ratings.dat', sep='::',engine='python',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # Entrenamos el algoritmo con el training dataset
    users = test.user.unique() # Devuelve un array con los users unicos en el testing dataset
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [2]:
def model_trainer(features,lrate,reg):
    all_recs = []
    test_data = []
    
    features=int(features)
    funksvd = FunkSVD(features=features, iterations=100, lrate=lrate, reg=reg, damping=5, range=None, bias=True) # define algorithm
      
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test) # save testing data
        all_recs.append(eval('FunkSVD', funksvd, train, test))
    
    all_recs = pd.concat(all_recs, ignore_index=True)
    test_data = pd.concat(test_data, ignore_index=True)
    rla1 = topn.RecListAnalysis()
    rla2 = topn.RecListAnalysis()
    rla1.add_metric(topn.precision)
    rla2.add_metric(topn.recall)
    prec = rla1.compute(all_recs, test_data)
    rec = rla2.compute(all_recs, test_data)
    F1 = ((prec['precision'].values*rec['recall'].values*2)/(prec['precision'].values+rec['recall'].values))
    F1 = np.nan_to_num(F1)
    return [F1.mean(),prec['precision'].mean(),rec['recall'].mean()]

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK

def objective(params):
    features = params['features']
    lrate = params['lrate']
    reg = params['reg']
    metric = model_trainer(features,lrate,reg)
    print([features,lrate,reg,metric[0],metric[1],metric[2]],',')
    return {'loss': -metric[0], 'status': STATUS_OK }

space={'features': hp.uniform('features', 1, 100),
       'lrate': hp.uniform('lrate', 0.0005, 0.005),
       'reg': hp.uniform('reg',0.005,0.05)}

best = fmin(objective, space, algo=tpe.suggest,max_evals=100)

[46.78813350877569, 0.002372567908402144, 0.008052472841875064, 0.04854337283120273, 0.03733443708609309, 0.13167548495796974]
,                                                                                                                      
[49.38817677586113, 0.0024158552924008424, 0.028820303633810346, 0.04461472182169803, 0.03467052980132474, 0.1159682135748965]
,                                                                                                                      
[75.74459076188624, 0.0029475039957984274, 0.020912569074875497, 0.04144424043240653, 0.032036423841059855, 0.11046165967107836]
,                                                                                                                      
[97.55477653698865, 0.003726701531014381, 0.006689806114623716, 0.03829901581964487, 0.029769867549669142, 0.1027259258046569]
,                                                                                                                      
[77.966985

KeyboardInterrupt: 

In [4]:
array = [ #100.46
    [19.18980896827522, 0.0020916172097523384, 0.007382122700112858, 0.045060456081449186, 0.03490066225165588, 0.12253758421731657]
,                                                                                                                      
[14.13124486376634, 0.004839058442812943, 0.019968934433925962, 0.03578323480867209, 0.02797516556291426, 0.09625342333139719]
,                                                                                                                      
[52.18773982982387, 0.004940089993276811, 0.028468976002549214, 0.039084295894371374, 0.0301572847682121, 0.10446154933269446]
,                                                                                                                      
[53.88307864005832, 0.0024364920879495575, 0.007525526746638337, 0.047350428572640435, 0.03640562913907317, 0.12727926503172268]
,                                                                                                                      
[19.724869266347863, 0.0039983537891302295, 0.029311291148779964, 0.04021761025112131, 0.03134437086092739, 0.10666570208231552]
,                                                                                                                      
[21.069169956179604, 0.0008521943592991593, 0.04696230142017912, 0.03476635297416181, 0.027801324503311478, 0.09036295080588574]
,                                                                                                                      
[92.68211807059775, 0.004561787607164464, 0.031598204184871435, 0.03746319681018616, 0.02886920529801353, 0.10120035025717874]
,                                                                                                                      
[37.101211220265185, 0.0031925617146827795, 0.01859099518980135, 0.04490830750205632, 0.03470695364238432, 0.11942827936314014]
,                                                                                                                      
[73.89621136526087, 0.001017755912181313, 0.024976692001284845, 0.027593548046888274, 0.02245198675496721, 0.0674175312824011]
,                                                                                                                      
[77.07097650846245, 0.0014455181729658368, 0.03412749859813296, 0.03169796556771272, 0.025294701986755194, 0.07779346913625643]
,                                                                                                                      
[25.48523677860519, 0.004035889681158587, 0.046596285991590276, 0.04428061146904548, 0.034011589403973766, 0.12021937244703215]
,                                                                                                                      
[93.63730300568885, 0.0024417116265544672, 0.03746237525251686, 0.03535441216317694, 0.027456953642384454, 0.09188291526149196]
,                                                                                                                      
[73.2905279540664, 0.0019718163058137722, 0.04951465077906712, 0.03285043751009348, 0.02573178807947047, 0.08435765369424526]
,                                                                                                                      
[45.278096978580855, 0.000982537583320916, 0.04337456722442884, 0.03298807116223568, 0.026420529801324727, 0.08404439513498081]
,                                                                                                                      
[76.79987958075137, 0.0036141902239944347, 0.014152578150279642, 0.040190666784108195, 0.030947019867550014, 0.10812778902570716]
,                                                                                                                      
[60.83517564831225, 0.004380177734568808, 0.006953864242012585, 0.03859492436213779, 0.02992880794702024, 0.10315654048949113]
,                                                                                                                      
[62.488293646009794, 0.00290291691766577, 0.01615156147425007, 0.04363237387837421, 0.03366721854304657, 0.11768845608807536]
,                                                                                                                      
[2.8869847687390746, 0.0034559324103749945, 0.026502669917630603, 0.05206738680221104, 0.039750000000000264, 0.1433928636103956]
,                                                                                                                      
[78.35480716702611, 0.0014610723460438384, 0.03726838114965889, 0.030359295816670274, 0.024223509933775155, 0.07415405837997731]
,                                                                                                                      
[17.46660229394246, 0.004630907965177969, 0.04708492654080736, 0.0401084873557711, 0.031140728476821394, 0.10697943943402886]
,                                                                                                                      
[1.5312765508952213, 0.0033692111927014837, 0.011607852990681886, 0.054773002836239516, 0.04093874172185466, 0.15686295717526808]
,                                                                                                                      
[3.3921167625383184, 0.003248532310628852, 0.02250556028466917, 0.049882399529068096, 0.038291390728477086, 0.13750315741943026]
,                                                                                                                      
[1.4327336523052447, 0.0034867578153946473, 0.011588811015241741, 0.05485757864498707, 0.041033112582781875, 0.15640461122258872]
,                                                                                                                      
[7.743905262160752, 0.003792492049715196, 0.011993793564127511, 0.04432893011452991, 0.03492218543046376, 0.1161917988184921]
,                                                                                                                      
[30.323191177478705, 0.0029020743415865847, 0.011287524716082593, 0.04745799027316669, 0.0363658940397353, 0.13008840391862544]
,                                                                                                                      
[9.796344527302038, 0.004156880016893508, 0.005102337716811612, 0.040302442106976304, 0.03177980132450355, 0.10622447097224316]
,                                                                                                                      
[36.39295606588025, 0.0026084618871720384, 0.011190316027043287, 0.048367144557994984, 0.037046357615894365, 0.13159408706400139]
,                                                                                                                      
[2.99237442302806, 0.0032115272846400342, 0.017241300744056554, 0.052951220521046026, 0.04031788079470226, 0.14474360379456253]
,                                                                                                                      
[28.477743137094972, 0.00207496694744353, 0.021943737946336365, 0.050175133925176474, 0.03894701986755002, 0.13215059027401233]
,                                                                                                                      
[12.187109857982833, 0.0036124814259841315, 0.008501261737697777, 0.03903499374594983, 0.030632450331126108, 0.10363632527260767]
,                                                                                                                      
[39.83444111794106, 0.0018147214408125983, 0.014516562001238992, 0.050199617344047924, 0.039107615894039965, 0.12960307384959974]
,                                                                                                                      
[1.104180733876147, 0.0005241499417895749, 0.01989546481018768, 0.049239326893480186, 0.037604304635761844, 0.13580883721062234]
,                                                                                                                      
[15.617755437621078, 0.002837621502818221, 0.005032970100472375, 0.03899607597306299, 0.030500000000000187, 0.1053504023923159]
,                                                                                                                      
[24.529689663618875, 0.00426326606686739, 0.02379823494087991, 0.04327008505280512, 0.03336589403973541, 0.11613550190456524]
,                                                                                                                      
[7.324912669998028, 0.0049692737101911105, 0.008977787385212323, 0.04182547337818222, 0.03298509933774847, 0.10913419570127202]
,                                                                                                                      
[46.96256394102178, 0.002331797505179471, 0.013588045803061345, 0.04747956073334437, 0.03682450331125867, 0.12514091598414248]
,                                                                                                                      
[32.263416586231884, 0.0038422272329379377, 0.030777338495781334, 0.043678506258522966, 0.03363907284768237, 0.1169161629155119]
,                                                                                                                      
[98.22064666580093, 0.004784172201301205, 0.0203220296858101, 0.0359228625345438, 0.027764900662252026, 0.09635355670984423]
,                                                                                                                      
[21.283406040461593, 0.0030539875435681833, 0.017254404231657573, 0.04668227255668292, 0.0358973509933777, 0.12699908485080738]
,                                                                                                                      
[56.1875807856605, 0.0034505025588723094, 0.02678058408648433, 0.041724518956004084, 0.0321937086092719, 0.11123356820148136]
,                                                                                                                      
[83.95173868869148, 0.002688395000483335, 0.0067006688761466, 0.04316732796065038, 0.033490066225165815, 0.11483942669814592]
,                                                                                                                      
[12.595270746050623, 0.004397967138698645, 0.01040952674931497, 0.03651973360284379, 0.02878145695364257, 0.09746064263783154]
,                                                                                                                      
[19.16501511325376, 0.0022559147508927676, 0.03423199312839343, 0.04450622811312694, 0.03480794701986779, 0.11717701495890231]
,                                                                                                                      
[67.39462205109818, 0.0016824676347916244, 0.028815320330823187, 0.039221735025804026, 0.03085099337748363, 0.09944540864377824]
,                                                                                                                      
[42.29940095709509, 0.003948909811119043, 0.041293956986107494, 0.04291444660075337, 0.033001655629139204, 0.11629040158255567]
,                                                                                                                      
[34.38603862778862, 0.002524903532598332, 0.015559714328542642, 0.04832170100803898, 0.0373394039735102, 0.12895970964969086]
,                                                                                                                      
[24.787743264097728, 0.0033367634032237265, 0.018865329454949172, 0.04572801690640916, 0.03521854304635802, 0.12395218199364867]
,                                                                                                                      
[54.60208466427755, 0.004651369227967963, 0.02425433600447547, 0.03860921861723028, 0.0298096026490069, 0.10394909254283585]
,                                                                                                                      
[5.857042358028739, 0.0037033874718837207, 0.012913994569735715, 0.0476779238177675, 0.036794701986755135, 0.1293514551411188]
,                                                                                                                      
[49.984507745172515, 0.00301701956945823, 0.00945703767586454, 0.04447130668115759, 0.03426655629139094, 0.12045216910149759]
,                                                                                                                      
[88.63814905987502, 0.0040839394447336605, 0.006483446330867431, 0.03879585120161959, 0.03012417218543073, 0.10373127661854024]
,                                                                                                                      
[13.735083376298839, 0.003487567687344724, 0.03420073272173709, 0.04116208354915931, 0.03224834437086115, 0.1085612984642596]
,                                                                                                                      
[23.0591732279598, 0.004463028318498204, 0.021423621462137873, 0.04227506013544812, 0.03275662251655655, 0.1125761621617524]
,                                                                                                                      
[68.44831763249199, 0.0011707095289171287, 0.030681045672271173, 0.031241149085398757, 0.025066225165563182, 0.07664726855641336]
,                                                                                                                      
[9.263885144091725, 0.0030787082429482686, 0.026074467311942025, 0.04594251810957332, 0.03596192052980156, 0.12137473163860783]
,                                                                                                                      
[29.400289282027966, 0.0022175558981155545, 0.0050983009918852875, 0.05205992502235168, 0.039895695364238815, 0.14094650344969237]
,                                                                                                                      
[17.393969933291455, 0.0027337949732159678, 0.017982152230012656, 0.04261426069624724, 0.033486754966887695, 0.11239444664214486]
,                                                                                                                      
[1.191471022653258, 0.00061013815117213, 0.015308120574162464, 0.05166094670114913, 0.03959437086092746, 0.13966342174243526]
,                                                                                                                      
[59.70159048429088, 0.003946668633376668, 0.04040125790373561, 0.04056859400238505, 0.031157284768212123, 0.10876298786789258]
,                                                                                                                      
[27.196728366027624, 0.004196172648784572, 0.02312322508582666, 0.04332989005635502, 0.03347682119205322, 0.11665755178714247]
,                                                                                                                      
[4.973325903487648, 0.0033610175697703654, 0.012650956976224191, 0.04861696093634542, 0.037567880794702295, 0.13085688289886427]
,                                                                                                                      
[38.185345641923504, 0.0018683456667805318, 0.036239984024794024, 0.04516882649909558, 0.03519039735099369, 0.1171241164441461]
,                                                                                                                      
[42.77532303108558, 0.0014223577136409393, 0.0493042523219367, 0.03780503362072335, 0.02978973509933799, 0.09702057855385723]
,                                                                                                                      
[10.615788867715636, 0.004758220378732308, 0.010247164181505468, 0.03857957908464359, 0.030369205298013454, 0.10137725239062542]
,                                                                                                                      
[50.79961855489765, 0.004304864118524155, 0.007199017960453886, 0.03920036739255646, 0.03039072847682147, 0.10524225798295676]
,                                                                                                                      
[1.4756227244600284, 0.0036522018700528226, 0.01666468123813377, 0.054388997489633016, 0.04064403973509967, 0.1545018179552967]
,                                                                                                                      
[15.872801238822086, 0.0036130726043700663, 0.016617597145732504, 0.03922793518164925, 0.030683774834437343, 0.10458593131629135]
,                                                                                                                      
[1.0201299310735061, 0.0038302079675586517, 0.02064137257242478, 0.053872848376945275, 0.04021357615894088, 0.15438665237698926]
,                                                                                                                      
[4.669983224693395, 0.003204027285805589, 0.014398514865889608, 0.04920465589054186, 0.03804304635761613, 0.13223676186956845]
,                                                                                                                      
[8.055351495754014, 0.002874184969169636, 0.011687734332945277, 0.04607841483967236, 0.03628807947019887, 0.12035245009997816]
,                                                                                                                      
[20.910749918850144, 0.003515376081687182, 0.00827858198763204, 0.044503429336477655, 0.03415728476821214, 0.12327661282811161]
,                                                                                                                      
[1.04147075450028, 0.004552876988197974, 0.006209478945167058, 0.05382049433887585, 0.04023841059602685, 0.15429529796225203]
,                                                                                                                      
[14.073054317515542, 0.0024362578222334695, 0.019562860889634444, 0.041296512250388905, 0.03254304635761614, 0.10906388267711813]
,                                                                                                                      
[32.061183943686956, 0.0037503844555811117, 0.01326579135490672, 0.04345014899776439, 0.033241721854305066, 0.11998605143937319]
,                                                                                                                      
[18.872060344125874, 0.004054021453833145, 0.02803301584197543, 0.04054626126862512, 0.031640728476821464, 0.10614103941307505]
,                                                                                                                      
[11.257620764608987, 0.0031385266103937225, 0.018281621628636304, 0.04235968571286028, 0.033266556291390845, 0.11274729974823]
,                                                                                                                      
[22.595046699280893, 0.003335328982464118, 0.010295122674662021, 0.045846542109232995, 0.03521523178807979, 0.1252461190765707]
,                                                                                                                      
[3.2625213117994143, 0.002938913020853507, 0.024720840728131275, 0.05057284076783903, 0.03883774834437104, 0.13698058934319504]
,                                                                                                                      
[7.213968652060627, 0.0028068512990978654, 0.015163173317515063, 0.0470579123546585, 0.03706953642384133, 0.1231528381810747]
,                                                                                                                      
[15.643154583743572, 0.004894211033599697, 0.016495005941355122, 0.03437349173699342, 0.026844370860927478, 0.09326377564797717],
    [46.78813350877569, 0.002372567908402144, 0.008052472841875064, 0.04854337283120273, 0.03733443708609309, 0.13167548495796974]
,                                                                                                                      
[49.38817677586113, 0.0024158552924008424, 0.028820303633810346, 0.04461472182169803, 0.03467052980132474, 0.1159682135748965]
,                                                                                                                      
[75.74459076188624, 0.0029475039957984274, 0.020912569074875497, 0.04144424043240653, 0.032036423841059855, 0.11046165967107836]
,                                                                                                                      
[97.55477653698865, 0.003726701531014381, 0.006689806114623716, 0.03829901581964487, 0.029769867549669142, 0.1027259258046569]
,                                                                                                                      
[77.96698552722074, 0.003283107974279712, 0.027003652584668924, 0.03977285004261828, 0.03062417218543081, 0.1075734930353229]
,                                                                                                                      
[52.71594234385462, 0.0018140353176387249, 0.019452739727749156, 0.04751194933688342, 0.037132450331126166, 0.12128975794613543]
,                                                                                                                      
[30.18846325706769, 0.0012917741624830664, 0.01340347725259388, 0.04978009881561158, 0.03906125827814591, 0.12897038247706932]
,                                                                                                                      
[24.75623665041253, 0.0018107738659989545, 0.015376084274465827, 0.050265459073275086, 0.03894701986754985, 0.13287261869199213]
,                                                                                                                      
[29.843097485345776, 0.002464017272460524, 0.0070022233892199585, 0.049965011570807606, 0.03831456953642418, 0.1360972813923435]
,                                                                                                                      
[2.943254440758102, 0.003906318663656627, 0.009946917456990223, 0.0523448643448399, 0.03993543046357641, 0.14329622303401815]
,                                                                                                                      
[26.404910431422124, 0.0031163227254226997, 0.04190819328869209, 0.04619978859452175, 0.035642384105960524, 0.12394250803164485]
,                                                                                                                      
[17.074940137114094, 0.0021701293357480273, 0.030450003540706597, 0.0434829025701084, 0.03427649006622528, 0.11341527025587352]
,                                                                                                                      
[57.14554247563224, 0.0036200485673512427, 0.030019982681355584, 0.04174618170280039, 0.032147350993377774, 0.11262953247840093]
,                                                                                                                      
[48.91857627122629, 0.004878915097959178, 0.024816420671490227, 0.03932132917289153, 0.030375827814569798, 0.1059410957305292]
,                                                                                                                      
[25.543253718740182, 0.0032287002380912483, 0.030398123729969244, 0.046055817195750144, 0.03546688741721877, 0.12275691566432377]
,                                                                                                                      
[43.618446504530134, 0.0029782356079905417, 0.019432064046627742, 0.04416395454668239, 0.03425827814569562, 0.11675419643424634]
,                                                                                                                      
[1.1618465563064038, 0.0034876806936233323, 0.033459972967901416, 0.05508640455980303, 0.0411092715231792, 0.15729205739421548]
,                                                                                                                      
[93.56649078873275, 0.0016648167751347309, 0.029666213576897527, 0.03195313518070907, 0.02534437086092747, 0.07854605786103018]
,                                                                                                                      
[1.2524287553991655, 0.004320443145541781, 0.03765626347069809, 0.05403229431890676, 0.040397350993377906, 0.15339707242316297]
,                                                                                                                      
[2.180258769286899, 0.0036441211051485164, 0.01023581512134574, 0.05236972173341642, 0.03992384105960284, 0.14365251049434768]
]

In [5]:
metric = pd.DataFrame(array)
metric.columns = ['features','lrate','reg','F1','Precision','Recall']
metric.head(5)

,features,lrate,reg,F1,Precision,Recall
0,19.189809,0.002092,0.007382,0.045060,0.034901,0.122538
1,14.131245,0.004839,0.019969,0.035783,0.027975,0.096253
2,52.187740,0.004940,0.028469,0.039084,0.030157,0.104462
3,53.883079,0.002436,0.007526,0.047350,0.036406,0.127279
4,19.724869,0.003998,0.029311,0.040218,0.031344,0.106666


In [6]:
metric.sort_values(by=['F1'],ascending=False).head(10)

,features,lrate,reg,F1,Precision,Recall
21,2.598613,0.003767,0.032516,0.075675,0.051315,0.230000
64,1.238649,0.004511,0.008495,0.075406,0.050848,0.231697
78,1.188406,0.003019,0.035712,0.075251,0.051060,0.224416
73,1.099824,0.002733,0.036609,0.075184,0.051007,0.222287
29,6.686745,0.002706,0.028429,0.073953,0.050785,0.213013
13,2.091639,0.004349,0.039564,0.073913,0.050021,0.228648
92,5.610414,0.003338,0.035461,0.073791,0.050530,0.215790
65,1.055413,0.004578,0.026308,0.073590,0.049926,0.218647
22,1.367495,0.004651,0.031911,0.073582,0.049873,0.222841
54,1.328269,0.003854,0.031004,0.073323,0.049692,0.219363


In [6]:
metric.to_csv('metric_F1.csv', index=False)

In [7]:
metric = pd.read_csv('C:\\Users\\Alejo\\Tesis\\1M\\FunkSVD\\results\\metric_F1.csv')
metric

,features,learning rate,reg,F1,Precision,Recall
0,19.189809,0.002092,0.007382,0.045060,0.034901,0.122538
1,14.131245,0.004839,0.019969,0.035783,0.027975,0.096253
2,52.187740,0.004940,0.028469,0.039084,0.030157,0.104462
3,53.883079,0.002436,0.007526,0.047350,0.036406,0.127279
4,19.724869,0.003998,0.029311,0.040218,0.031344,0.106666
...,...,...,...,...,...,...
95,43.618447,0.002978,0.019432,0.044164,0.034258,0.116754
96,1.161847,0.003488,0.033460,0.055086,0.041109,0.157292
97,93.566491,0.001665,0.029666,0.031953,0.025344,0.078546
98,1.252429,0.004320,0.037656,0.054032,0.040397,0.153397


In [8]:
%matplotlib qt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
colormap = cm.hot
x = metric['reg']
y = metric['learning rate']
z = metric['features'] 
c = metric['F1']


fig = plt.figure()

ax = fig.add_subplot(111, projection='3d')
img = ax.scatter(x, y, z, c=c, cmap=plt.hsv())
ax.set_xlabel('reg')
ax.set_ylabel('learning rate')
ax.set_zlabel('features')
fig.colorbar(img)
plt.show()